In [128]:
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, fbeta_score

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE


In [ ]:
X_train = pd.read_csv("data/split/X_train.csv")
y_train = pd.read_csv("data/split/y_train.csv")["is_fraud"]

X_test = pd.read_csv("data/split/X_test.csv")
y_test = pd.read_csv("data/split/y_test.csv")["is_fraud"]

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)


Train: (800, 13) (800,)
Test : (200, 13) (200,)


In [ ]:
print("y_train distribution:")
print(y_train.value_counts())

print("\ny_test distribution:")
print(y_test.value_counts())


y_train distribution:
is_fraud
0    760
1     40
Name: count, dtype: int64

y_test distribution:
is_fraud
0    190
1     10
Name: count, dtype: int64


In [ ]:
# Cette cellule sert à séparer les variables numériques
# des variables catégorielles afin d'appliquer
# le bon preprocessing à chaque type de données.
numeric_features = ["montant", "score_risque_client"]
categorical_features = ["devise", "canal", "type_transaction"]


In [ ]:
# Pipeline de preprocessing des variables numériques
# =========================================
# Cette pipeline est appliquée uniquement aux colonnes numériques.
# Elle permet :
# 1) de gérer les valeurs manquantes
# 2) de standardiser les données pour les modèles basés sur la distance 

numeric_transformer = Pipeline(steps=[
    # Étape 1 : Imputation des valeurs manquantes
    # Remplace les NaN par la moyenne de chaque colonne
    # La moyenne est calculée uniquement sur les données d'entraînement
    ("imputer", SimpleImputer(strategy="mean")),
    # Étape 2 : Standardisation
    # Centre les données (moyenne = 0) et réduit l'écart-type à 1
    # Indispensable pour les modèles sensibles à l'échelle des variables
    ("scaler", StandardScaler())
])

In [ ]:
# Pipeline de preprocessing des variables catégorielles
# =========================================
# Cette pipeline est appliquée uniquement aux colonnes catégorielles.
# Elle permet :
# 1) de gérer les valeurs manquantes
# 2) de transformer les catégories en variables numériques (One-Hot Encoding)

categorical_transformer = Pipeline(steps=[
    # Étape 1 : Imputation des valeurs manquantes
    # Remplace les NaN par la catégorie la plus fréquente
    # Calculée uniquement sur les données d'entraînement
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])


In [ ]:
# Préprocesseur global : combinaison des pipelines
# numériques et catégorielles
# ======================================================
# Ce ColumnTransformer applique :
# - le pipeline numérique aux colonnes numériques
# - le pipeline catégoriel aux colonnes catégorielles
# Chaque type de donnée reçoit le traitement adapté,
# sans mélange et sans data leakage.
preprocessor = ColumnTransformer(
    transformers=[
        # (nom, pipeline_de_traitement, colonnes_ciblées)
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [ ]:
# Pipeline complète : preprocessing → SMOTE → features sélection
#  → modèle de classification
# ==========================================================
# Cette pipeline garantit :
# - aucun data leakage
# - un traitement cohérent train / test
# - une amélioration du rappel (F2-score)
# - une compatibilité avec le déploiement
pipeline = Pipeline(steps=[
    # Étape 1 : Preprocessing global
    # - imputation
    # - standardisation
    # - encodage One-Hot
    ("preprocessing", preprocessor),
    # Étape 2 : Rééquilibrage des classes (TRAIN uniquement)
    # SMOTE génère des exemples synthétiques de la classe minoritaire
    # pour corriger le déséquilibre des données
    ("smote", SMOTE(random_state=42)),
    # Étape 3 : Sélection des meilleures features
    # Garde uniquement les 5 variables les plus pertinentes
    # selon le test statistique ANOVA (f_classif)
    ("feature_selection", SelectKBest(score_func=f_classif, k=5)),
    # Étape 4 : Modèle de classification
    # Logistic Regression :
    # - modèle interprétable
    # - adapté aux données déséquilibrées
    ("model", LogisticRegression(
        max_iter=1000,             # Assure la convergence du modèle
        class_weight="balanced"    # Donne plus de poids à la classe minoritaire
    ))
])


In [136]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['montant',
                                                   'score_risque_client']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['devise', 'canal',
                                                   'type_transaction'])])),
                ('smote', SMOTE(random_state=42)),
                ('feature_selection', SelectKBest(k=5)),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [ ]:
# Évaluation du modèle avec le seuil par défaut (0.5)
# =========================================

# Prédictions binaires sur le jeu de test
# Le seuil par défaut utilisé par predict() est 0.5
y_pred_default = pipeline.predict(X_test)

print("=== Évaluation classique (threshold = 0.5) ===")
# Rapport de classification :
# - precision
# - recall
# - f1-score
# - support
print(classification_report(y_test, y_pred_default))
# Calcul du F2-score :
# Le F2 favorise le rappel (détection de la fraude)
print("F2-score :", fbeta_score(y_test, y_pred_default, beta=2))


=== Évaluation classique (threshold = 0.5) ===
              precision    recall  f1-score   support

           0       0.97      0.53      0.69       190
           1       0.07      0.70      0.13        10

    accuracy                           0.54       200
   macro avg       0.52      0.62      0.41       200
weighted avg       0.93      0.54      0.66       200

F2-score : 0.25735294117647056


In [ ]:
# Le pipeline applique toutes les transformations apprises sur le train
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Seuil de décision personnalisé
threshold = 0.4 #  0.3 / 0.2 / 0.15 / 0.1

# Conversion des probabilités en classes binaires
# Si P(fraude) >= threshold → fraude (1), sinon non fraude (0)
y_pred_tuned = (y_proba >= threshold).astype(int)

print("=== Évaluation avec threshold =", threshold, "===")
print(classification_report(y_test, y_pred_tuned))
print("F2-score :", fbeta_score(y_test, y_pred_tuned, beta=2))


=== Évaluation avec threshold = 0.4 ===
              precision    recall  f1-score   support

           0       0.96      0.23      0.37       190
           1       0.05      0.80      0.10        10

    accuracy                           0.26       200
   macro avg       0.50      0.51      0.23       200
weighted avg       0.91      0.26      0.35       200

F2-score : 0.20512820512820512


In [170]:
from sklearn.neighbors import KNeighborsClassifier

# =========================================
# Test du modèle KNN
# =========================================

pipeline_knn = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("feature_selection", SelectKBest(score_func=f_classif, k=5)),
        ("model", KNeighborsClassifier(
            n_neighbors=7,      # nombre de voisins
            weights="distance"  # donne plus de poids aux voisins proches
        ))
    ]
)

# Entraînement
pipeline_knn.fit(X_train, y_train)

# Probabilités
y_proba_knn = pipeline_knn.predict_proba(X_test)[:, 1]

# Seuil orienté recall
threshold = 0.5
y_pred_knn = (y_proba_knn >= threshold).astype(int)

# Évaluation
print("=== KNN | threshold =", threshold, "===")
print(classification_report(y_test, y_pred_knn))
print("F2-score :", fbeta_score(y_test, y_pred_knn, beta=2))


=== KNN | threshold = 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.81      0.89       190
           1       0.18      0.80      0.29        10

    accuracy                           0.81       200
   macro avg       0.58      0.80      0.59       200
weighted avg       0.95      0.81      0.86       200

F2-score : 0.47058823529411764


In [165]:
from xgboost import XGBClassifier

# =========================================
# Test du modèle XGBoost
# =========================================

pipeline_xgb = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("feature_selection", SelectKBest(score_func=f_classif, k=5)),
        ("model", XGBClassifier(
            n_estimators=300,
            max_depth=5,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric="logloss",
            random_state=42,
            n_jobs=-1
        ))
    ]
)

# Entraînement
pipeline_xgb.fit(X_train, y_train)

# Probabilités
y_proba_xgb = pipeline_xgb.predict_proba(X_test)[:, 1]

# Seuil orienté recall
threshold = 0.4
y_pred_xgb = (y_proba_xgb >= threshold).astype(int)

# Évaluation
print("=== XGBoost | threshold =", threshold, "===")
print(classification_report(y_test, y_pred_xgb))
print("F2-score :", fbeta_score(y_test, y_pred_xgb, beta=2))


=== XGBoost | threshold = 0.4 ===
              precision    recall  f1-score   support

           0       0.99      0.77      0.86       190
           1       0.15      0.80      0.26        10

    accuracy                           0.77       200
   macro avg       0.57      0.78      0.56       200
weighted avg       0.94      0.77      0.83       200

F2-score : 0.43478260869565216


In [166]:
joblib.dump(pipeline_knn, "pipeline.joblib")
print("Pipeline KNN sauvegardé avec succès ")


Pipeline KNN sauvegardé avec succès 
